# ¿Hay valores atípicos en el conjunto de datos?


In [1]:
# Librerias
import pandas as pd
import yaml
import numpy as np

In [3]:
# Archivo Dataset Final
df = pd.read_excel("/content/Final_DF.xlsx")

# Verificar dimensiones
df.shape


(168, 524)

In [4]:
vars_numericas = df.select_dtypes(include=["int64", "float64"]).columns.tolist()
df_numerico = df[vars_numericas]

In [5]:
Q1 = df_numerico.quantile(0.25)
Q3 = df_numerico.quantile(0.75)
IQR = Q3 - Q1

outliers_count = ((df_numerico < (Q1 - 1.5 * IQR)) |
                  (df_numerico > (Q3 + 1.5 * IQR))).sum()

In [6]:
outliers_df = pd.DataFrame({
    "variable": outliers_count.index,
    "num_outliers": outliers_count.values
}).sort_values(by="num_outliers", ascending=False)

outliers_df.head(10)

,variable,num_outliers
384,ffq_croquetas,79
180,ipaq_nivel_af,61
13,edo_civil,60
503,pgh_perturbaciones,47
318,ffq_pistacho,41
356,ffq_j_serrano,41
385,ffq_sopasobre,40
17,ejercicio,40
26,alergias_alim,40
243,cv_p4,40


In [13]:
outliers_df["num_outliers"].sum()

np.int64(4648)

**Conclusión**

El análisis exploratorio de valores atípicos mediante el criterio del rango intercuartílico (IQR) permitió identificar un total de 4,648 observaciones extremas, distribuidas en 389 variables del conjunto de datos. Este resultado refleja la alta dimensionalidad del dataset y la heterogeneidad inherente a un estudio longitudinal con múltiples mediciones por paciente.

Es importante destacar que una proporción considerable de los valores identificados como atípicos corresponde a variables categóricas u ordinales codificadas numéricamente, tales como frecuencias de consumo alimentario, niveles de actividad física y escalas de cuestionarios clínicos.

En estos casos, la noción de outlier desde un punto de vista estrictamente estadístico no implica necesariamente la presencia de errores, sino que puede representar variabilidad real y clínicamente relevante entre individuos y visitas.

En esta etapa del proyecto, el análisis de outliers tiene un carácter descriptivo y exploratorio, por lo que no se aplicaron técnicas de eliminación ni transformación de valores. El tratamiento específico de estos valores será considerado en fases posteriores, en función del objetivo analítico y del contexto científico de cada variable. Para efectos de visualización, se reportan únicamente las variables con mayor número de valores atípicos, aunque el análisis se realizó sobre la totalidad de las variables numéricas disponibles.

# ¿Cuál es la cardinalidad de las variables categóricas?

Identificación y cardinalidad de variables categóricas
Como parte del análisis exploratorio de datos (EDA), se realizó la identificación de las variables categóricas presentes en el conjunto de datos, así como el cálculo de su cardinalidad. Este análisis permite comprender la estructura del dataset, la diversidad de categorías y la complejidad asociada al tratamiento posterior de dichas variables.

Las variables fueron clasificadas en tres grandes grupos:

* **Variables categóricas nominales**, que representan categorías sin un orden inherente (por ejemplo, estado civil, presencia de alergias).
* **Variables categóricas ordinales**, que corresponden a escalas con un orden lógico o jerárquico (por ejemplo, niveles de actividad física, frecuencias de consumo).
* **Variables numéricas**, que representan magnitudes cuantitativas continuas o discretas.
Para las variables categóricas, se calculó la cardinalidad como el número de valores únicos observados.

Este análisis es particularmente relevante en un contexto longitudinal, ya que un número elevado de categorías puede reflejar variabilidad entre pacientes y entre visitas, y no necesariamente inconsistencias en los datos.

In [14]:
# Ver tipos de datos
df.dtypes.value_counts()

,count
float64,303
int64,207
object,14


**Variables numéricas y categóricas**

In [15]:
# Variables tipo object (categóricas claras)
cat_object = df.select_dtypes(include=["object"]).columns.tolist()

# Variables numéricas
num_vars = df.select_dtypes(include=["int64", "float64"]).columns.tolist()

print("Variables categóricas (object):", len(cat_object))
print("Variables numéricas:", len(num_vars))

Variables categóricas (object): 14
Variables numéricas: 510


Aunque inicialmente las variables fueron clasificadas según su tipo de dato en el entorno de programación (por ejemplo, int64, float64 u object), esta clasificación no refleja necesariamente su naturaleza estadística. Por ello, se realizó una revisión conceptual de las variables numéricas codificadas, identificando aquellas que corresponden a variables categóricas ordinales (por ejemplo, escalas, frecuencias de consumo y clasificaciones clínicas).

Como resultado, del total de 510 variables numéricas técnicas, 22 fueron reclasificadas como categóricas ordinales, manteniéndose 488 como variables numéricas reales.

**Identificar variables categóricas ordinales**

In [17]:
# Cardinalidad por variable numérica
cardinality = df[num_vars].nunique()

# Umbral ajustable (ej. ≤ 10 categorías)
ordinal_candidates = cardinality[cardinality <= 10].sort_values()

ordinal_candidates.head(20)

,0
ffq_ac_maiz,1
bilirrubina_orina,1
cetonas_orina,1
glucosa_orina,1
albumina_orina,1
menst_flag,2
periodo,2
grupo,2
int_quirugica,2
fam_cardiacos,2


In [18]:
# Cardinalidad de variables categóricas object
cat_cardinality = df[cat_object].nunique().sort_values(ascending=False)

cat_cardinality

,0
tmb,152
pgh_h_levantarse,28
pgh_h_acostarse,20
leucocitos_sed,9
hematies_sed,6
bacterias_sed,5
celulas_epiteliales_sed,5
oxalato_calcico_sed,5
fibra_mucoide_sed,4
num_depo,3


In [19]:
summary_cardinality = pd.DataFrame({
    "variable": cat_cardinality.index,
    "cardinality": cat_cardinality.values
})

summary_cardinality.head(10)

,variable,cardinality
0,tmb,152
1,pgh_h_levantarse,28
2,pgh_h_acostarse,20
3,leucocitos_sed,9
4,hematies_sed,6
5,bacterias_sed,5
6,celulas_epiteliales_sed,5
7,oxalato_calcico_sed,5
8,fibra_mucoide_sed,4
9,num_depo,3


**Conclusión**

Como parte del análisis exploratorio de datos, se realizó una caracterización de las variables según su tipo y cardinalidad. El conjunto de datos está compuesto mayoritariamente por variables numéricas (510), acompañadas de un número reducido de variables categóricas tipo object (14).

No obstante, se identificó que un subconjunto relevante de las variables numéricas corresponde en realidad a variables categóricas ordinales, las cuales se encuentran codificadas como valores enteros. Estas incluyen variables binarias (por ejemplo, consumo de alcohol, uso de suplementos, presencia/ausencia de compuestos en orina), así como clasificaciones clínicas y de cuestionarios, donde existe un orden lógico entre las categorías pero no una distancia cuantitativa estricta entre ellas.

El análisis de cardinalidad de las variables categóricas tipo object mostró una marcada heterogeneidad. Algunas variables presentan una cardinalidad elevada, como tmb (152 categorías), lo cual sugiere información de tipo descriptivo o texto libre, mientras que otras presentan cardinalidades bajas o moderadas, asociadas principalmente a mediciones clínicas y observaciones de laboratorio.

En esta etapa del proyecto, la identificación de la tipología y cardinalidad de las variables permite comprender mejor la estructura del dataset y sienta las bases para decisiones metodológicas posteriores, como la codificación adecuada de variables, la posible reducción de dimensionalidad y la selección de técnicas analíticas acordes a la naturaleza de los datos. No se realizaron transformaciones ni recodificaciones en esta fase, dado el enfoque exploratorio y descriptivo del análisis.

# ¿Hay correlación entre las variables dependientes e independientes?

En esta etapa del proyecto no se realizó un análisis formal de correlación entre variables independientes y dependientes, debido a que el objetivo principal corresponde a un análisis exploratorio orientado a la comprensión de la estructura, calidad y características generales del conjunto de datos.

Adicionalmente, el estudio presenta una estructura longitudinal, con múltiples observaciones por paciente a lo largo del tiempo, lo que implica dependencia entre las mediciones. La estimación directa de correlaciones sin considerar esta dependencia podría conducir a interpretaciones sesgadas o incorrectas.

Dado que el propósito del proyecto es la generación de conocimiento científico más que la optimización de un modelo predictivo, el análisis de relaciones entre variables se plantea como una etapa posterior, una vez definido el enfoque metodológico adecuado para tratar la variabilidad intra-sujeto y la dimensión temporal del estudio.